In [3]:
import random 
import numpy as np
import copy
import re
import sys
import json

In [4]:
# filename = sys.argv[1]
# fp = open(filename,"r", encoding='utf8')
fp = open("perceptron-training-data/train-labeled.txt","r", encoding='utf8')
lines = fp.readlines()
vocabulary = {}
data = {}

i = 0
for line in lines:
    line_list = line.split() 
    reality = 1 if (line_list[1] == "True") else -1
    sentiment = 1 if (line_list[2] == "Pos") else -1
    data[i] = {"reality":reality, "sentiment":sentiment, "sentence": line_list[3:]}
    for word in data[i]["sentence"]:
        copy = word
        word = word.replace(".","")
        word = word.replace(",","")
        word = word.replace("...","")
        word = word.replace(":","")
        word = word.replace(";","")
        word = word.replace("\'","")
        word = word.replace("\"","")
        word = word.replace("!","")
        word = word.replace("?","")
        word = word.replace("(","")
        word = word.replace(")","")
        word = word.replace("~","")
        word = word.replace("-","")
        
        word = word.lower()
        if(word == ""):
            data[i]["sentence"].remove(copy)
            continue
        
        index = data[i]["sentence"].index(copy)
        data[i]["sentence"][index] = word
        
        if(word in vocabulary):
            vocabulary[word] += 1
        else:
            vocabulary[word] = 1
        
    i += 1
        
vocabulary = dict(sorted(vocabulary.items(), key=lambda x: x[1], reverse = True)) #first 34

vocab_vector = []
for key in vocabulary.keys():
    if(vocabulary[key] > 5 and vocabulary[key] < 900):
        vocab_vector.append(key)

In [5]:
def line_to_vec(line):
    blank = [0] * len(vocab_vector)
    for i in line:
        if(i in vocab_vector):
            index = vocab_vector.index(i)
            blank[index] += 1
            
    return blank

In [6]:
for i in data.keys():
    data[i]["vector"] = line_to_vec(data[i]["sentence"])

Vanilla Model

In [7]:
def vanilla_model(category):
    w = [random.random() for i in range(len(vocab_vector))]
    b = random.random()
    iterations = 25
    for iteration in range(iterations):
        for i in data.keys():
            a = np.sum(np.multiply(w, data[i]["vector"])) + b
            
            a = 1 if(a>0) else -1
            y = data[i][category]
            
            if(y * a <= 0):
                w = np.add(w, np.multiply(data[i]["vector"], y))
                b += y
    return (w, b)

reality, b_r = vanilla_model("reality")
sentiment, b_s = vanilla_model("sentiment")

Average Model

In [8]:
# def average_model(category):
#     iterations = 25
#     w_a = [0] * len(vocab_vector)
#     b_a = 0
#     w = [random.random() for i in range(len(vocab_vector))]
#     b = random.random()
#     for iteration in range(iterations):
#         for i in data.keys():
#             a = np.sum(np.multiply(w, data[i]["vector"]))
            
#             a = 1 if(a>0) else -1
#             y = data[i][category]
            
#             if(y * a < 0):
#                 w = np.add(w, np.multiply(data[i]["vector"], y))
#                 b += y
                
#         w_a = np.add(w_a,w)
#         b_a += b
#     return (np.divide(w_a, iterations) , b_a/iteration)

# reality_avg, b_r_a = average_model("reality")
# sentiment_avg, b_s_a = average_model("sentiment")
# print(reality_avg)

In [9]:
def average_model(category):
    iterations = 25
    w_a = [0] * len(vocab_vector)
    b_a = 0
    w = [random.random() for i in range(len(vocab_vector))]
    b = random.random()
    for iteration in range(iterations):
        for i in data.keys():
            a = np.sum(np.multiply(w, data[i]["vector"]))
            
            a = 1 if(a>0) else -1
            y = data[i][category]
            
            if(y * a < 0):
                w = np.add(w, np.multiply(data[i]["vector"], y))
                b += y
                
                w_a = np.add(w_a, np.multiply(data[i]["vector"], y*(iteration+1)))
                b_a += b
                
    return (np.subtract(w, np.divide(w_a, iterations)) , b - (b_a/iteration))

reality_avg, b_r_a = average_model("reality")
sentiment_avg, b_s_a = average_model("sentiment")
print(reality_avg)

[-16.53772901  -1.1398463    2.05810482 ...   2.89067187   5.20975627
   3.15233136]


In [45]:
vanilla = {"vocab": vocab_vector, "reality":list(reality),"bias_r":b_r, "sentiment":list(sentiment), "bias_s":b_s}
average = {"vocab": vocab_vector, "reality":list(reality_avg), "bias_r":b_r_a, "sentiment":list(sentiment_avg), "bias_s":b_s_a}
with open("vanillamodel.txt","w") as file:
    file.write(json.dumps(vanilla)) 
with open("averagedmodel.txt","w") as file:
    file.write(json.dumps(average)) 

In [ ]:
#Vanila - 0.84154
#Averaged - 0.84524

In [10]:
input_f = "perceptron-training-data/dev-text.txt"
fp = open(input_f, "r", encoding='utf8')
f = open("percepoutput.txt", "w", encoding='utf8')
lines = fp.readlines()
for line in lines[:8]:
    sentence = line.strip().split()
    id = sentence[0]
    sentence = sentence[1:]
    word_vector = line_to_vec(sentence)
    
    a = np.multiply(word_vector, reality_avg)
    reality_result = "True" if np.sum(a)+b_r > 0 else "Fake"
    
    a = np.multiply(word_vector, sentiment_avg)
    sentiment_result = "Pos" if np.sum(a)+b_s > 0 else "Neg"
    
    print(""+id+" "+reality_result+" "+sentiment_result+"\n")
    
f.close()

0EL4s2q Fake Neg

0UEraLY Fake Neg

0cKBWf6 True Neg

0gQlq5g Fake Neg

0lsa78x Fake Neg

0oy1qqz Fake Pos

1Eonp7p Fake Neg

1acV0r8 Fake Pos

